# add addl USCDI element extensions and tags to SD

1. get list of data elements that are add'l USCDI
2. read SD from resources-yaml folder
3. create a copy
4. find elements based on the addl USCDI elements list
   1. add USCDI extension
   2. add add'l USCDI tag ( in unicode ) to short
      1. if short does not exist, add short from base resource - todo - not needed for this set
   3. sort keys in element
   4. save copy 

In [1]:
from yaml import load as yload, dump as ydump, Loader
from json import loads, dumps
from pathlib import Path
from copy import deepcopy
from datetime import datetime
from fhir.resources.structuredefinition import StructureDefinition

in_path = Path(r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml')
R4_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package')

out_path = Path(r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-copy')
#create out_path using pathlib Path.mkdir(parents=True, exist_ok=True)
out_path.mkdir(parents=True, exist_ok=True)

today = datetime.today().strftime('%Y-%m-%d')
today, in_path, out_path

('2023-10-11',
 PosixPath('/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml'),
 PosixPath('/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-copy'))

In [2]:
# get csv file of add'l uscdi elements using pandas
import pandas as pd
df = pd.read_csv(r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/data/additional-uscdi-requirements.csv')

#list the FHIR Element column
addl_uscdi_elements = df['FHIR Element'].tolist()
addl_uscdi_elements = [x.split(' or ') for x in addl_uscdi_elements[1:]]
addl_uscdi_elements = [item for sublist in addl_uscdi_elements for item in sublist]
# addl_uscdi_elements = list(set(addl_uscdi_elements))
addl_uscdi_elements

['Patient.telecom',
 'Patient.communication',
 'Patient.extension:race',
 'Patient.extension:ethnicity',
 'Patient.extension:tribalAffiliation',
 'Patient.extension:sex',
 'Patient.extension:genderIdentity',
 'Patient.deceased[x]',
 'Patient.address.use',
 'Patient.address.period',
 'Patient.name.use',
 'Patient.name.period',
 'Patient.name.suffix',
 'ServiceRequest.reasonCode',
 'ServiceRequest.reasonReference',
 'MedicationRequest.reasonCode',
 'MedicationRequest.reasonReference',
 'MedicationRequest.extension:medicationAdherence',
 'Procedure.basedOn',
 'DocumentReference.category:uscore',
 'Observation.derivedFrom',
 'Specimen.collection.bodySite',
 'Specimen.condition']

In [3]:
# read StructureDefinitions from resources-yaml folder using pathlib
for sd in in_path.glob('Struct*.yml'):
    print(f"***********{sd}***********")
    r = yload(sd.read_text(), Loader=Loader)
    r_copy = deepcopy(r)
    print(f"☞ ☞ ☞ Type = {r['type']}")
    for i, element in enumerate(r['differential']['element']):
        try:
            print(f"☞ ☞ ☞ element {i}: {element['id']}")
            if element['id'] in addl_uscdi_elements:
                print('❗❗❗add the USCDI extension')
                r_copy['differential']['element'][i]["extension"] = [{
                    "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-uscdi",
                    "valueBoolean": True
                }]
                if 'short' in element.keys():
                    print("❗❗❗add the '𝗔𝗗𝗗'𝗟 𝗨𝗦𝗖𝗗𝗜:' tag to short")
                    r_copy['differential']['element'][i]["short"] = f"𝗔𝗗𝗗'𝗟 𝗨𝗦𝗖𝗗𝗜: {element['short']}"
                else:
                    print("❗❗❗add the short element from the base")

            print('❗❗❗ done with element')
            print('-'*10)

        except Exception as e:
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
            print('-'*10)

    print('❗❗❗ done with SD')
    print('-'*40)
    print('\n')

    # custom sort the keys in the elements using fhir.resources
    r_copy = StructureDefinition.parse_obj(r_copy)


    # save the copy to out_path using Pathlib
    copyfile = out_path / sd.name
    copyfile.write_text(r_copy.yaml(indent=2))

***********/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml/StructureDefinition-us-core-specimen.yml***********
☞ ☞ ☞ Type = Specimen
☞ ☞ ☞ element 0: Specimen
❗❗❗ done with element
----------
☞ ☞ ☞ element 1: Specimen.identifier
❗❗❗ done with element
----------
☞ ☞ ☞ element 2: Specimen.accessionIdentifier
❗❗❗ done with element
----------
☞ ☞ ☞ element 3: Specimen.type
❗❗❗ done with element
----------
☞ ☞ ☞ element 4: Specimen.subject
❗❗❗ done with element
----------
☞ ☞ ☞ element 5: Specimen.collection
❗❗❗ done with element
----------
☞ ☞ ☞ element 6: Specimen.collection.bodySite
❗❗❗add the USCDI extension
❗❗❗add the '𝗔𝗗𝗗'𝗟 𝗨𝗦𝗖𝗗𝗜:' tag to short
❗❗❗ done with element
----------
☞ ☞ ☞ element 7: Specimen.condition
❗❗❗add the USCDI extension
❗❗❗add the '𝗔𝗗𝗗'𝗟 𝗨𝗦𝗖𝗗𝗜:' tag to short
❗❗❗ done with element
----------
❗❗❗ done with SD
----------------------------------------


***********/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml/StructureDefinition-us-core

In [4]:
##### DO THIS ONLY when you want to start over with removing all tags ####

# !cp /Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-copy/Struct*.yml /Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml